In [2]:
import cv2
import numpy as np
from pathlib import Path

def check_mask_values(mask_dir="../unet_data/train/masks"):
    mask_dir = Path(mask_dir)
    
    if not mask_dir.exists():
        print(f"Directory {mask_dir} not found")
        return
    
    mask_files = list(mask_dir.glob("*.png"))
    
    if not mask_files:
        print("No mask files found")
        return
    
    print(f"Checking {len(mask_files)} mask files...")
    
    for i, mask_file in enumerate(mask_files[:3]):  # Check first 3 masks
        mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)
        
        unique_values = np.unique(mask)
        print(f"\nMask {mask_file.name}:")
        print(f"  Shape: {mask.shape}")
        print(f"  Unique values: {unique_values}")
        print(f"  Min: {mask.min()}, Max: {mask.max()}")
        print(f"  Non-zero pixels: {np.count_nonzero(mask)}")
        
        # Show distribution
        for val in unique_values:
            count = np.sum(mask == val)
            percentage = (count / mask.size) * 100
            class_name = "background" if val == 0 else f"class_{val}"
            print(f"    {class_name} (value={val}): {count} pixels ({percentage:.1f}%)")

def visualize_mask(mask_path, output_path=None):
    """Create a visualization of the mask with scaled values"""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Scale values for visualization (0->0, 1->85, 2->170, 3->255, etc.)
    unique_vals = np.unique(mask)
    if len(unique_vals) > 1:
        max_class = unique_vals[-1]
        if max_class > 0:
            mask_vis = (mask * (255 // max_class)).astype(np.uint8)
        else:
            mask_vis = mask
    else:
        mask_vis = mask
    
    if output_path:
        cv2.imwrite(output_path, mask_vis)
        print(f"Visualization saved to {output_path}")
    
    return mask_vis


 

In [3]:
print("=== Mask Values Check ===")
check_mask_values()

# Create a visualization example
mask_dir = Path("../unet_data/train/masks")
if mask_dir.exists():
    mask_files = list(mask_dir.glob("*.png"))
    if mask_files:
        first_mask = mask_files[0]
        vis_path = str(first_mask).replace('.png', '_visualization.png')
        visualize_mask(str(first_mask), vis_path)
        print(f"\nCreated visualization: {vis_path}")

=== Mask Values Check ===
Checking 51 mask files...

Mask tomato-012_IMG_8319.png:
  Shape: (3024, 4032)
  Unique values: [0 2 3 4 5 6 9]
  Min: 0, Max: 9
  Non-zero pixels: 830689
    background (value=0): 11362079 pixels (93.2%)
    class_2 (value=2): 5299 pixels (0.0%)
    class_3 (value=3): 199724 pixels (1.6%)
    class_4 (value=4): 5015 pixels (0.0%)
    class_5 (value=5): 356058 pixels (2.9%)


    class_6 (value=6): 258435 pixels (2.1%)
    class_9 (value=9): 6158 pixels (0.1%)

Mask tomato-040_IMG_8551.png:
  Shape: (3024, 4032)
  Unique values: [0 2 3 4 5 6 8 9]
  Min: 0, Max: 9
  Non-zero pixels: 436348
    background (value=0): 11756420 pixels (96.4%)
    class_2 (value=2): 1306 pixels (0.0%)
    class_3 (value=3): 151037 pixels (1.2%)
    class_4 (value=4): 1026 pixels (0.0%)
    class_5 (value=5): 185140 pixels (1.5%)
    class_6 (value=6): 70292 pixels (0.6%)
    class_8 (value=8): 25833 pixels (0.2%)
    class_9 (value=9): 1714 pixels (0.0%)

Mask tomato-030_IMG_8466.png:
  Shape: (3024, 4032)
  Unique values: [0 4 9]
  Min: 0, Max: 9
  Non-zero pixels: 577320
    background (value=0): 11615448 pixels (95.3%)
    class_4 (value=4): 15235 pixels (0.1%)
    class_9 (value=9): 562085 pixels (4.6%)
Visualization saved to ../unet_data/train/masks/tomato-012_IMG_8319_visualization.png

Created visualization: ../unet_data/train/masks/tomato-012_IMG_8319_visualization.png


In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def create_colored_mask_visualization():
    """Create colored visualizations of masks"""
    mask_dir = Path("../unet_data/train/masks")
    output_dir = Path("../mask_visualizations")
    output_dir.mkdir(exist_ok=True)
    
    if not mask_dir.exists():
        print("Mask directory not found")
        return
    
    # Color palette for classes
    colors = [
        [0, 0, 0],       # background - black
        [255, 0, 0],     # class 1 - red  
        [0, 255, 0],     # class 2 - green
        [0, 0, 255],     # class 3 - blue
        [255, 255, 0],   # class 4 - yellow
        [255, 0, 255],   # class 5 - magenta
        [0, 255, 255],   # class 6 - cyan
    ]
    
    mask_files = list(mask_dir.glob("*.png"))[:5]  # First 5 masks
    
    for mask_file in mask_files:
        mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)
        
        # Create colored visualization
        colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
        
        unique_values = np.unique(mask)
        for val in unique_values:
            if val < len(colors):
                colored_mask[mask == val] = colors[val]
        
        # Save colored mask
        output_path = output_dir / f"colored_{mask_file.name}"
        cv2.imwrite(str(output_path), colored_mask)
        
        print(f"Created colored visualization: {output_path}")
        print(f"  Classes found: {unique_values}")


create_colored_mask_visualization()

Created colored visualization: ../mask_visualizations/colored_tomato-012_IMG_8319.png
  Classes found: [0 2 3 4 5 6 9]
Created colored visualization: ../mask_visualizations/colored_tomato-040_IMG_8551.png
  Classes found: [0 2 3 4 5 6 8 9]
Created colored visualization: ../mask_visualizations/colored_tomato-030_IMG_8466.png
  Classes found: [0 4 9]
Created colored visualization: ../mask_visualizations/colored_tomato-001_image2.png
  Classes found: [0 2 3 4 5 6 8 9]
Created colored visualization: ../mask_visualizations/colored_tomato-030_IMG_8467.png
  Classes found: [0 9]


In [6]:
from PIL import Image

img = Image.open('/root/workspace/few_shot_data/unet/train_2shot_seed3/masks/tomato-012_IMG_8320.png')

In [7]:
np.unique(img) 

array([0, 1, 3, 5, 6, 8], dtype=uint8)